In [ ]:
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd
import math
import os
from scipy import sparse, stats

from matplotlib import pyplot as plt
SEED = 923984

In [ ]:
from reservoir.activation_functions import tanh, heaviside, sigmoid

activation_function = lambda x : tanh(x)

# Loading and Preprocessing

Lots of different on availabale : https://towardsdatascience.com/a-data-lakes-worth-of-audio-datasets-b45b88cd4ad

Regression : http://tseregression.org/ + https://arxiv.org/pdf/2012.02974

Prediction Datasets available :

* MackeyGlass
* Lorenz
* Sunspot

Classification Datasets available :

* Custom :  FSDD, HAART, JapaneseVowels
* Aeon : SpokenArabicDigits, CatsDogs, LSST
* Torchaudio: SPEECHCOMMANDS

More on https://www.timeseriesclassification.com/dataset.php or https://pytorch.org/audio/stable/datasets.html

In [ ]:

#Preprocessing
from datasets.multivariate_generation import generate_multivariate_dataset, extract_peak_frequencies
from sklearn.preprocessing import MinMaxScaler
from datasets.preprocessing import scale_data
from datasets.preprocessing import add_noise, duplicate_data
from datasets.load_data import load_data as load_dataset
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit, StratifiedGroupKFold
from datasets.preprocessing import flexible_indexing

# Define noise parameter
noise_std = 0.001

data_type = "normal" # "normal" ou "noisy"

def load_data(dataset_name, data_type, noise_std, step_ahead=5, visualize=False):
    (is_instances_classification, is_multivariate, sampling_rate,
     X_train_raw, X_test_raw, Y_train_raw, Y_test,
     use_spectral_representation, spectral_representation,
     groups) = load_dataset(dataset_name, step_ahead, visualize=False)
    
    WINDOW_LENGTH = 10
    freq_train_data = X_train_raw
    flat_train_data = np.concatenate(freq_train_data, axis=0) if is_instances_classification else freq_train_data
    extract_peak_frequencies(flat_train_data, sampling_rate, smooth=True, window_length=WINDOW_LENGTH, threshold=1e-5, nperseg=1024, visualize=False)

    if is_multivariate:
        X_train_band, X_test_band = X_train_raw, X_test_raw
        del X_train_raw, X_test_raw
        X_val_band = None
    else:
        X_test, X_train = X_test_raw, X_train_raw
        X_val, X_val_band = None, None
        del X_train_raw, X_test_raw
    Y_train = Y_train_raw
    del Y_train_raw

            
    # PREPROCESSING
    hop = 50 if is_instances_classification else 1
    win_length = edge_cut = 100
    if not is_multivariate:
        X_train_band = generate_multivariate_dataset(
            X_train, is_instances_classification, spectral_representation, hop=hop, win_length = win_length
        )
        
        X_test_band = generate_multivariate_dataset(
            X_test, is_instances_classification, spectral_representation, hop=hop, win_length = win_length
        )
    elif not use_spectral_representation:
        X_train_band = generate_multivariate_dataset(
            X_train_band, is_instances_classification, spectral_representation, hop=hop, win_length = win_length
        )
        X_test_band = generate_multivariate_dataset(
            X_test_band, is_instances_classification, spectral_representation, hop=hop, win_length = win_length
        )
    else:
        print("Data is already spectral and multivariate, nothing to do")

        
    if not is_instances_classification:
        X_train_band = X_train_band[edge_cut:-edge_cut] 
        X_test_band = X_test_band[edge_cut:-edge_cut] 
        
        Y_train = Y_train[edge_cut:-edge_cut] 
        Y_test  = Y_test[edge_cut:-edge_cut]
    
    scaler_multi = MinMaxScaler(feature_range=(0, 1))
    X_train_band, X_val_band, X_test_band = scale_data(X_train_band, X_val_band, X_test_band, scaler_multi, is_instances_classification)
                
    if not is_multivariate:
        scaler_x_uni = MinMaxScaler(feature_range=(0, 1))
        X_train, X_val, X_test = scale_data(X_train, X_val, X_test, scaler_multi, is_instances_classification)       
    
    # NOISE
    if data_type == "noisy":
        if is_instances_classification:
            # UNI
            if not is_multivariate:
                X_train_noisy = [add_noise(instance, noise_std) for instance in tqdm(X_train, desc="TRAIN")]
                X_test_noisy = [add_noise(instance, noise_std) for instance in tqdm(X_test, desc="TEST")]
                
            # MULTI
            X_train_band_noisy = [add_noise(instance, noise_std) for instance in tqdm(X_train_band, desc="TRAIN")]
            X_test_band_noisy = [add_noise(instance, noise_std) for instance in tqdm(X_test_band, desc="TEST")]
        
        else:  #if prediction
            # UNI
            if not is_multivariate:
                X_train_noisy = add_noise(X_train, noise_std)
                X_test_noisy = add_noise(X_test, noise_std)
        
            # MULTI
            X_train_band_noisy = add_noise(X_train_band, noise_std)
            X_test_band_noisy = add_noise(X_test_band, noise_std)
    
    # Define the number of instances you want to select
    if is_instances_classification:
        num_samples_for_pretrain = 500 if len(X_train_band) >= 500 else len(X_train_band)
        indices = np.random.choice(len(X_train_band), num_samples_for_pretrain, replace=False)
    else:
        indices = range(len(X_train_band))
    
    
    if data_type == "noisy":
        # Defining pretrain   
        if not is_multivariate:
            X_pretrain_noisy = np.array(X_train_noisy, dtype=object)[indices].flatten()
        X_pretrain_band_noisy = np.array(X_train_band_noisy, dtype=object)[indices]
    
    if not is_multivariate:
        X_pretrain = np.array(X_train, dtype=object)[indices].flatten()
    X_pretrain_band = np.array(X_train_band, dtype=object)[indices]

    return X_pretrain_band, X_train_band, X_test_band, Y_train, Y_test, is_multivariate, is_instances_classification

# Dataset characteristics

In [ ]:
datasets = [
    "CatsDogs", "FSDD", "JapaneseVowels", "SPEECHCOMMANDS", "SpokenArabicDigits"
]

for dataset_name in datasets:
    (is_instances_classification, is_multivariate, sampling_rate,
     X_train_raw, X_test_raw, Y_train_raw, Y_test,
     use_spectral_representation, spectral_representation,
     groups) = load_dataset(dataset_name, 5, visualize=False)
    
    # Compute the length of each instance (assuming each instance is a 1D sequence)
    lengths = [len(x) for x in X_train_raw]
    avg_length = sum(lengths) / len(lengths)
    
    print(f"Dataset: {dataset_name}  -->  Average length of X_train_raw: {avg_length:.2f}")

In [ ]:

# Define the dataset list
datasets = [
    "MackeyGlass", "Lorenz", "Sunspot_daily", "Henon", "NARMA", "CatsDogs", 
    "FSDD", "JapaneseVowels", "SPEECHCOMMANDS", "SpokenArabicDigits"
]

# Function to test dataset characteristics
def dataset_characteristics(dataset, data_type, noise_std):
    pretrain_data, train_data, test_data, Y_train, Y_test, is_multivariate, is_instances_classification = load_data(dataset, data_type, noise_std, visualize=False)

    # Score for prediction
    if dataset == "Sunspot":
        start_step = 30
        end_step = 500
    else:
        start_step = 500
        end_step = 1500
    SLICE_RANGE = slice(start_step, end_step)

    common_index = 1
    if is_instances_classification:
        common_size = pretrain_data[0].shape[common_index]
    else:
        common_size = pretrain_data.shape[common_index]

    # We want the size of the reservoir to be at least network_size
    network_size = 500
    K = math.ceil(network_size / common_size)
    n = common_size * K

    # Pretraining dataset processing
    if is_instances_classification:
        unsupervised_pretrain = np.concatenate(pretrain_data).astype(float)
    else:
        unsupervised_pretrain = pretrain_data.astype(float)

    # **Calculate total step count**
    if is_instances_classification:
        total_pretrain_length = sum(instance.shape[0] for instance in pretrain_data)
        total_train_length = sum(instance.shape[0] for instance in train_data)
        total_test_length = sum(instance.shape[0] for instance in test_data)
        original_dimension = train_data[0].shape[1]
        number_class = Y_test.shape[1]
        
        # Calculate min, max and average sample size over test+train datasets
        sample_sizes = [instance.shape[0] for instance in train_data] + [instance.shape[0] for instance in test_data]
        min_sample_size = min(sample_sizes)
        max_sample_size = max(sample_sizes)
        avg_sample_size = sum(sample_sizes) / len(sample_sizes)
    else:
        total_pretrain_length = pretrain_data.shape[0]
        total_train_length = train_data.shape[0]
        total_test_length = test_data.shape[0]
        original_dimension = train_data.shape[1]
        number_class = 0
        min_sample_size = None
        max_sample_size = None
        avg_sample_size = None

    return {
        "dataset_name": dataset,
        "tot_pretrain_length": total_pretrain_length,
        "tot_train_length": total_train_length,
        "tot_test_length": total_test_length,
        "origin_dim": original_dimension,
        "nb_dupication": K,
        "reservoir_size": n,
        "final_dim": common_size,
        "nb_class": number_class,
        "min_sample_size": min_sample_size,
        "max_sample_size": max_sample_size,
        "avg_sample_size": avg_sample_size,
    }


# Run tests on all datasets
results = []
for dataset in datasets:
    characteristics = dataset_characteristics(dataset, data_type, noise_std)
    results.append(characteristics)

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Save to CSV
csv_filename = "outputs/dataset_characteristics.csv"
df.to_csv(csv_filename, index=False)

print(f"Dataset characteristics saved to {csv_filename}")

# Evaluation function

In [ ]:
from scipy import stats
from numpy import random

# Evaluating
from performances.esn_model_evaluation import train_model_for_classification, predict_model_for_classification, compute_score
from performances.esn_model_evaluation import (train_model_for_prediction, init_reservoir, init_ip_reservoir, init_local_rule_reservoir, 
                                                init_ip_local_rule_reservoir, init_readout)
from analysis.richness import spectral_radius, pearson, squared_uncoupled_dynamics_alternative, distance_correlation
from reservoir.reservoir import init_matrices
from connexion_generation.hag import run_algorithm


nb_jobs = 10
def evaluate_dataset_on_test(study, dataset_name, function_name, pretrain_data, train_data, test_data, Y_train, Y_test, is_instances_classification, nb_trials = 8, record_metrics=False):
    # Collect all hyperparameters in a dictionary
    hyperparams = {param_name: param_value for param_name, param_value in study.best_trial.params.items()}
    print(hyperparams)
    leaky_rate = 1
    input_connectivity = 1

    # score for prediction
    if dataset_name == "Sunspot":
        start_step = 30
        end_step = 500
    else:
        start_step = 500
        end_step = 1500
    SLICE_RANGE = slice(start_step, end_step)

    if 'variance_target' not in hyperparams and 'min_variance' in hyperparams:
        hyperparams['variance_target'] = hyperparams['min_variance']
    if not is_instances_classification:
        hyperparams['use_full_instance'] = False

    RIDGE_COEF = 10**hyperparams['ridge']
    
    if function_name in ["hadsp", "desp"]:
        max_partners = np.inf
    
    scores = [] 
    if record_metrics:
        spectral_radii = []
        pearson_correlations = []
        CEVs = []
        dcors = []
    for i in range(nb_trials):
        common_index = 1
        if is_instances_classification:
            common_size = pretrain_data[0].shape[common_index]
        else:
            common_size = pretrain_data.shape[common_index]

        # We want the size of the reservoir to be at least network_size
        K = math.ceil(hyperparams['network_size'] / common_size)
        n = common_size * K
        
        if function_name in ["diag_ee", "diag_ei"]:
            use_block = True
        else:
            use_block = False
            
        # UNSUPERVISED PRETRAINING 
        if function_name == "random_ee":
            Win, W, bias = init_matrices(n, input_connectivity, hyperparams['connectivity'],  K, w_distribution=stats.uniform(loc=0, scale=1), use_block=use_block, seed=random.randint(0, 1000))
        else:
            Win, W, bias = init_matrices(n, input_connectivity, hyperparams['connectivity'],  K, w_distribution=stats.uniform(loc=-1, scale=2), use_block=use_block, seed=random.randint(0, 1000))
        bias *= hyperparams['bias_scaling']
        Win *= hyperparams['input_scaling']

        if function_name == "hadsp":
            W, (_, _, _) = run_algorithm(W, Win, bias, hyperparams['leaky_rate'], activation_function, pretrain_data, 
                                     hyperparams['weight_increment'], hyperparams['target_rate'], hyperparams['rate_spread'], function_name, 
                                     multiple_instances=is_instances_classification, 
                                     min_increment = hyperparams['min_increment'], max_increment=hyperparams['max_increment'], use_full_instance=hyperparams['use_full_instance'],
                                     max_partners=max_partners, method="pearson", n_jobs=nb_jobs)
        elif function_name == "desp":
            W, (_, _, _) = run_algorithm(W, Win, bias, hyperparams['leaky_rate'], activation_function, pretrain_data, 
                                         hyperparams['weight_increment'], hyperparams['variance_target'], hyperparams['variance_spread'], function_name, 
                                         multiple_instances=is_instances_classification, 
                                         min_increment = hyperparams['min_increment'], max_increment=hyperparams['max_increment'], use_full_instance = hyperparams['use_full_instance'], 
                                         max_partners=max_partners, method = "pearson", 
                                         intrinsic_saturation=hyperparams['intrinsic_saturation'], intrinsic_coef=hyperparams['intrinsic_coef'], 
                                         n_jobs = nb_jobs)
        elif function_name in ["random_ee", "random_ei", "diag_ee", "diag_ei", "ip_correct", "anti-oja_fast", "ip-anti-oja_fast"]:
            eigen = sparse.linalg.eigs(W, k=1, which="LM", maxiter=W.shape[0] * 20, tol=0.1, return_eigenvectors=False)
            W *= hyperparams['spectral_radius'] / max(abs(eigen))
        else:
            raise ValueError(f"Invalid function: {function_name}")
        
        # unsupervised local rules
        if is_instances_classification:
            unsupervised_pretrain = np.concatenate(pretrain_data).astype(float)
        else:
            unsupervised_pretrain = pretrain_data.astype(float)
        if function_name == "ip_correct":
            reservoir = init_ip_reservoir(W, Win, bias, mu=hyperparams['mu'], sigma=hyperparams['sigma'], learning_rate=hyperparams['learning_rate'],
                                          leaking_rate=hyperparams['leaky_rate'], activation_function=activation_function
                                          )
            _ = reservoir.fit(unsupervised_pretrain, warmup=100)
        elif function_name == "anti-oja_fast":
            reservoir = init_local_rule_reservoir(W, Win, bias, local_rule="anti-oja", eta=hyperparams['oja_eta'],
                                                  synapse_normalization=False, bcm_theta=None,
                                                  leaking_rate=hyperparams['leaky_rate'], activation_function=activation_function,
                                                  )
            _ = reservoir.fit(unsupervised_pretrain, warmup=100)        
        elif function_name == "ip-anti-oja_fast":
            reservoir = init_ip_local_rule_reservoir(W, Win, bias, local_rule="anti-oja", eta=hyperparams['oja_eta'],
                                                      synapse_normalization=False, bcm_theta=None,
                                                      mu=hyperparams['mu'], sigma=hyperparams['sigma'], learning_rate=hyperparams['learning_rate'],
                                                      leaking_rate=hyperparams['leaky_rate'], activation_function=activation_function,
                                                      )
            _ = reservoir.fit(unsupervised_pretrain, warmup=100)
        else:
            reservoir = init_reservoir(W, Win, bias, leaky_rate, activation_function)
        readout = init_readout(ridge_coef=RIDGE_COEF)

        
        # TRAINING and EVALUATION
        if record_metrics:
            inputs = np.concatenate(test_data, axis=0) if is_instances_classification else test_data
            states_history_multi = reservoir.run(inputs)
            
            sr = spectral_radius(W)
            pearson_correlation, _ = pearson(states_history_multi, num_windows=1, size_window=len(states_history_multi), step_size = 1, show_progress=False)
            CEV = squared_uncoupled_dynamics_alternative(states_history_multi, num_windows=1, size_window=len(states_history_multi), step_size = 1, show_progress=True)
            dcor = distance_correlation(states_history_multi, num_windows=1, size_window=len(states_history_multi), step_size = 1, show_progress=True, method="auto", nb_jobs=nb_jobs)

            spectral_radii.append(sr)
            pearson_correlations.append(pearson_correlation[0])
            CEVs.append(CEV[0])
            dcors.append(dcor[0])
        else:
            if is_instances_classification:
                mode = "sequence-to-vector"
                train_model_for_classification(reservoir, readout, train_data, Y_train, n_jobs = nb_jobs, mode=mode)
    
                Y_pred = predict_model_for_classification(reservoir, readout, test_data, n_jobs = nb_jobs, mode=mode)
                score = compute_score(Y_pred, Y_test, is_instances_classification)
            else:
                esn = train_model_for_prediction(reservoir, readout, train_data, Y_train, warmup=start_step, n_jobs = nb_jobs)
                
                Y_pred =  esn.run(test_data, reset=False)
                score = compute_score(Y_pred[SLICE_RANGE], Y_test[SLICE_RANGE], is_instances_classification)
    
            scores.append(score)
            
    if record_metrics:
        return spectral_radii, pearson_correlations, CEVs, dcors

    return scores



# Common visualisation definitions

In [ ]:
from seaborn import color_palette

# -- Define color palettes for each group --
blues = color_palette("Blues", 5)      # shades of blue
oranges = color_palette("Oranges", 2)  # shades of orange
greens = color_palette("Greens", 2)    # shades of green
reds = color_palette("Reds", 2)    # shades of yellow

# -- Map each function to its color --
function_colors = {
    'E-ESN':                greens[0],
    'ESN':                  greens[1],
    'IP':                   blues[0],
    'Anti-Oja':             blues[1],
    'Anti-Oja-Fast':        blues[2],
    'IP +\nAnti-Oja':       blues[3],
    'IP +\nAnti-Oja\nFast': blues[4],
    'mean HAG':             oranges[0],
    'variance HAG':         oranges[1],
#    'diag EE':             reds[0],
#    'diag EI':             reds[1],
}

# If you want a specific order for the bars, you can enforce it:
functions_order = [
    'E-ESN',
    'ESN',
    'IP',
    'Anti-Oja',
    'IP +\nAnti-Oja',
    'mean HAG',
    'variance HAG',
#    'diag EE',
#    'diag EI',
]

function_mapping = {
    'ip-anti-oja_fast': 'IP +\nAnti-Oja',
    'ip_correct':       'IP',
    'anti-oja_fast':    'Anti-Oja',
    'desp':             'variance HAG',
    'hadsp':            'mean HAG',
    'random_ei':        'ESN',
    'random_ee':        'E-ESN',
#    'diag_ei':        'diag EI',
#    'diag_ee':        'diag EE',
}

# HPO Comparaison

In [ ]:
# Read your CSV file (already containing TPE vs. CMA-ES results)
file_name = "outputs/hpo_strategy.csv"
df = pd.read_csv(file_name)

# --- 1) Basic Cleanup ---
# Remove '%' from "Average Score" so we can convert to float
df['Average Score'] = pd.to_numeric(
    df['Average Score'].str.replace('%', ''), 
    errors='coerce'
)
# Remove '±' and '%' from "Standard Deviation"
df['Standard Deviation'] = pd.to_numeric(
    df['Standard Deviation'].str.replace('±', '').str.replace('%', ''), 
    errors='coerce'
)


# Optionally remove an unwanted function
# df = df[df['Function'] != 'ip']

# Rename functions for your final labels
df['Function'] = df['Function'].replace(function_mapping)

# Optional replacements for dataset names
df['Dataset'] = df['Dataset'].replace({
    'JapaneseVowels':     'Japanese\nVowels',
})


# 2b) distinguish TPE vs CMA-ES by hatch pattern (or alpha, edgecolor, etc.)
sampler_hatching = {
    'tpe':   '',     # no hatch
    'cmaes': '///',  # diagonal hatch
}

# Filter to only those in the DataFrame
functions = [f for f in functions_order if f in df['Function'].unique()]

# If you also want an explicit order for Sampler
samplers = ['tpe', 'cmaes']

# --- 3) Build the grouped bar chart ---

datasets = df['Dataset'].unique()
datasets.sort()  # optional: sort datasets alphabetically
x = np.arange(len(datasets))  # label locations
width = 0.08                  # narrower bar for more sub-bars

fig, ax = plt.subplots(figsize=(12, 6))
fontsize = 12

# Outer loop over functions
for i, func in enumerate(functions):
    # Inner loop over the two samplers
    for j, sampler in enumerate(samplers):
        # Horizontal offset for each bar
        # We have 2 samplers per function, so total sub-width = 2*width
        offset = i * (2 * width) + j * width

        # Extract rows for this function + sampler
        sub_df = df[(df['Function'] == func) & (df['Sampler'] == sampler)]

        # Merge with the dataset ordering to align bar positions
        merged = pd.DataFrame({'Dataset': datasets}).merge(sub_df, on='Dataset', how='left')

        # Plot the bars
        ax.bar(
            x + offset,
            merged['Average Score'],
            width,
            label=None,  # We'll manually handle legend
            yerr=merged['Standard Deviation'],
            capsize=4,
            color=function_colors.get(func, 'gray'),  # fallback color if missing
            hatch=sampler_hatching.get(sampler, ''),  # or '' if missing
            edgecolor='black'                         # optional to see the hatch better
        )

# --- 4) Cosmetic adjustments ---
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='both', labelsize=fontsize)

# X-axis label
ax.set_xlabel('Dataset', fontsize=fontsize)

# Y-axis label (change if classification vs. NRMSE)
# e.g., if your CSV is for classification, you might put "Classification Rate (%)"
ax.set_ylabel('Average Score', fontsize=fontsize)

# Position x-ticks in the center of each dataset group
total_functions = len(functions)
total_samplers = len(samplers)  # 2
group_width = total_functions * total_samplers * width
ax.set_xticks(x + group_width/2 - (width/2))
ax.set_xticklabels(datasets, rotation=0)

# --- 5) Build a custom legend ---
# 5a) Legend for the Functions (colors)
from matplotlib.patches import Patch

function_legend = [
    Patch(facecolor=function_colors[f], edgecolor='black', label=f) for f in functions
]

# 5b) Legend for the Samplers (hatching)
sampler_legend = [
    Patch(facecolor='white', edgecolor='black', hatch=sampler_hatching[s], 
          label=s.upper())  # or s.title()
    for s in samplers
]

# Combine them in one line, or do them separately
first_legend = ax.legend(handles=function_legend, title='Function', loc='upper left', fontsize=fontsize)
ax.add_artist(first_legend)  # explicitly add the first legend, then a second one
ax.legend(handles=sampler_legend, title='Sampler', loc='lower left', fontsize=fontsize)

plt.tight_layout()
plt.show()

# Test scores

In [ ]:
import numpy as np

from datetime import datetime

from performances.utility import retrieve_best_model

# Create an empty DataFrame to store the results
columns = ['Dataset', 'Function', 'Average Score', 'Standard Deviation', 'Date']
variate_type = "multi"  # "multi" or "uni"

for dataset_name in ["SPEECHCOMMANDS"]:
    new_results = pd.DataFrame(columns=columns)
    # Can be "MackeyGlass", "Lorenz", "Sunspot_daily", "CatsDogs", "JapaneseVowels", "SpokenArabicDigits", "FSDD", "SPEECHCOMMANDS"
    pretrain_data, train_data, test_data, Y_train, Y_test, is_multivariate, is_instances_classification = load_data(dataset_name, data_type, noise_std, visualize=True)
    if is_instances_classification:
        file_name = "test_results/test_results_classification.csv"
    else: 
        file_name = "test_results/test_results_prediction.csv"
    print(dataset_name)
    # Simulate your data and loop for evaluation
    
    # "random_ee", "random_ei", "diag_ee", "diag_ei", "ip_correct", "anti-oja_fast",  "ip-anti-oja_fast", "hadsp", "desp"
    for function_name in ["ip_correct", "hadsp", "desp"]:
        print(function_name)
        study = retrieve_best_model(function_name, dataset_name, is_multivariate, variate_type = "multi", data_type = "normal")
    
        scores = evaluate_dataset_on_test(
            study, 
            dataset_name,
            function_name, 
            pretrain_data, 
            train_data, 
            test_data,
            Y_train, 
            Y_test,
            is_instances_classification,
            nb_trials = 8,
            record_metrics=False
        )
        # Compute the average and standard deviation of the scores
        average_score = np.mean(scores)
        std_deviation = np.std(scores)
    
        if is_instances_classification:
            formatted_average = f"{round(average_score * 100, 5)} %"
            formatted_std = f"± {round(std_deviation * 100, 5)} %"
        else:
            formatted_average = f"{round(average_score, 5)}"
            formatted_std = f"± {round(std_deviation, 5)}"
        
        # Capture the current date
        current_date = datetime.now().strftime('%Y-%m-%d')
        
        # Create a new DataFrame row with the Date column
        new_row = pd.DataFrame({
            'Dataset': [dataset_name],
            'Function': [function_name],
            'Average Score': [formatted_average],
            'Standard Deviation': [formatted_std],
            'Date': [current_date]
        })
        
        # Concatenate the new row to the results DataFrame
        new_results = pd.concat([new_results, new_row], ignore_index=True)
    
    
    # Display the DataFrame
    print(new_results)
    
    # Load the existing CSV
    if os.path.exists(file_name):
        previous_results = pd.read_csv(file_name)
    else:
        columns = ['Dataset', 'Function', 'Average Score', 'Standard Deviation', 'Date']
        previous_results = pd.DataFrame(columns=columns)
        previous_results.to_csv(file_name, index=False)
        print(f"{file_name} created successfully.")
        
    tots_results = pd.concat([new_results, previous_results], axis=0)
    
    tots_results.to_csv(file_name, index=False)
    print(f"Results saved to {file_name}.")

## Visualisation

In [ ]:
import os
import pandas as pd 

file_name = "test_results/test_results_classification.csv"

if 'file_name' not in locals() and 'file_name' not in globals():
    file_name = "test_results/test_results_prediction.csv"  #  test_results_classification.csv or test_results_prediction.csv

if os.path.exists(file_name):
    previous_results = pd.read_csv(file_name)
else:
    # File does not exist, create it with the necessary columns
    columns = ['Dataset', 'Function', 'Average Score', 'Standard Deviation', 'Date']
    previous_results = pd.DataFrame(columns=columns)
    # Save the empty DataFrame as a CSV
    previous_results.to_csv(file_name, index=False)
    print(f"{file_name} created successfully.")

print(f"Results saved to {file_name}.")
previous_results

In [ ]:
import pandas as pd

from matplotlib import pyplot as plt
import numpy as np

all_results = pd.read_csv(file_name)
df = pd.DataFrame(all_results)

# Clean data as before
df['Average Score'] = df['Average Score'].astype(str).str.replace('%', '').astype(float)
df['Standard Deviation'] = df['Standard Deviation'].str.replace('±', '').str.replace('%', '').astype(float)

df = df[df['Function'] != 'ip']

df['Function'] = df['Function'].map(function_mapping)

# Optional replacements for dataset names
#df = df[df['Dataset'].isin(["Lorenz", "MackeyGlass", "Sunspot"])]

if file_name == "test_results/test_results_classification.csv":
    df['Dataset'] = df['Dataset'].str.replace('SpokenArabicDigits', 'Spoken\nArabic\nDigits')
    df['Dataset'] = df['Dataset'].str.replace('SPEECHCOMMANDS', 'SPEECH\nCOMMANDS')
    df['Dataset'] = df['Dataset'].str.replace('JapaneseVowels', 'Japanese\nVowels')


fig, ax = plt.subplots(figsize=(10, 6))

datasets = df['Dataset'].unique()
x = np.arange(len(datasets))  # The label locations
width = 0.1                   # Width of each bar

for i, func in enumerate(functions_order):
    # Grab only rows for this function
    values = df[df['Function'] == func]
    
    # We create a Series in the same order as 'datasets'
    merged = pd.DataFrame({'Dataset': datasets}).merge(values, on='Dataset', how='left')
    
    ax.bar(
        x + i * width,
        merged['Average Score'],
        width,
        label=func,
        yerr=merged['Standard Deviation'],
        capsize=5,
        color=function_colors[func]
    )

fontsize = 14

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='both', labelsize=fontsize)

if file_name == "test_results/test_results_prediction.csv":
    plt.ylabel('NRMSE', size=fontsize)
else:
    plt.ylabel('Classification Rate', size=fontsize)
plt.legend(title='Algorithm', fontsize=10, title_fontsize=fontsize, bbox_to_anchor=(0.5, 1.15), loc='upper center', ncol=len(functions_order))

# Position x-ticks in the center of all the bars for each dataset
ax.set_xticks(x + width * (len(functions_order)-1)/2)
ax.set_xticklabels(datasets)

plt.tight_layout()
plt.show()

# Export best parameters

In [ ]:
import os
import optuna
import pandas as pd
from performances.utility import camel_to_snake, retrieve_best_model


# List of datasets (extract from filenames)
datasets = [
    "SpokenArabicDigits",
    "JapaneseVowels",
    "FSDD",
    "SPEECHCOMMANDS",
    "CatsDogs",
    "MackeyGlass",
    "Lorenz",
    "Sunspot_daily",
]

for function_name in ["random_ee", "random_ei", "ip_correct", "anti-oja_fast",  "ip-anti-oja_fast", "hadsp", "desp"]:
    results = []
    for dataset in datasets:
        study = retrieve_best_model(function_name, dataset, is_multivariate=True, variate_type = "multi", data_type = "normal")
        best_trial = study.best_trial
        results.append({
            "dataset": dataset,
            "function_name": function_name,
            **best_trial.params,
        })
    # Convert results to a DataFrame
    df = pd.DataFrame(results)
    
    # Save to CSV
    df.to_csv(f"outputs/best_hyperparameters_{function_name}.csv", index=False)
    
    print(f"Results saved to best_hyperparameters_{function_name}.csv")


# Richness

In [ ]:
import os
import optuna
import pandas as pd
from performances.utility import camel_to_snake, retrieve_best_model

# Create an empty DataFrame to store the results
columns = [
    "dataset", 
    "function_name", 
    "spectral_radius_mean", 
    "spectral_radius_std", 
    "pearson_mean", 
    "pearson_std",
    "CEV_mean",
    "CEV_std",
    "dcor_mean",
    "dcor_std",
    "final_correlations_mean",
    "final_correlations_std",
]


# List of datasets (extract from filenames)
datasets = [
#    "JapaneseVowels",
#    "CatsDogs",
#    "FSDD",
#   "SpokenArabicDigits",
    "SPEECHCOMMANDS",
#    "MackeyGlass",
#    "Lorenz",
#    "Sunspot_daily",
]


new_results = pd.DataFrame(columns=columns)
for dataset in datasets:
    print(dataset)
    pretrain_data, train_data, test_data, Y_train, Y_test, is_multivariate, is_instances_classification = load_data(dataset, data_type, noise_std, visualize=False)
    for function_name in ["ip_correct", "anti-oja_fast",  "ip-anti-oja_fast", "hadsp", "desp"]: # "random_ee", "random_ei", "ip_correct", "anti-oja", "anti-oja_fast",  "ip-anti-oja", "hadsp", "desp"
        # Get the best trial from the study
        print(function_name)
        study = retrieve_best_model(function_name, dataset, is_multivariate, variate_type = "multi", data_type = "normal")
        
        SRs, pearsons, CEVs, dcors = evaluate_dataset_on_test(
            study, 
            dataset,
            function_name, 
            pretrain_data, 
            train_data, 
            test_data,
            Y_train, 
            Y_test,
            is_instances_classification,
            nb_trials = 1,
            record_metrics=True
        )
        # Create a new DataFrame row
        new_row = pd.DataFrame({
            "dataset": [dataset],
            "function_name": [function_name],
            "spectral_radius_mean": [np.mean(SRs)],
            "spectral_radius_std": [np.std(SRs)],
            "pearson_mean": [np.mean(pearsons)],
            "pearson_std": [np.std(pearsons)],
            "CEV_mean": [np.mean(CEVs)],
            "CEV_std": [np.std(CEVs)],
            "dcor_mean": [np.mean(dcors)],
            "dcor_std": [np.std(dcors)],
        })
    
        # Concatenate the new row to the results DataFrame
        new_results = pd.concat([new_results, new_row], ignore_index=True)
        

# Display the DataFrame
print(new_results)



In [ ]:
print(new_results)
file_name = "outputs/metrics.csv"

# Load the existing CSV
if os.path.exists(file_name) and os.path.getsize(file_name) > 0:
    try:
        previous_results = pd.read_csv(file_name)
    except pd.errors.EmptyDataError:
        print(f"{file_name} is empty. Initializing with default columns.")
        previous_results = pd.DataFrame(columns=columns)
        previous_results.to_csv(file_name, index=False)
else:
    print(f"{file_name} does not exist or is empty. Creating a new file.")
    previous_results = pd.DataFrame(columns=columns)
    previous_results.to_csv(file_name, index=False)
    
tots_results = pd.concat([new_results, previous_results], axis=0)

tots_results.to_csv(file_name, index=False)

print(f"Results saved to {file_name}")

## Visualize

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.ticker import MaxNLocator  # Importing here for completeness

# Load your data
file_name = 'outputs/metrics.csv'
data = pd.read_csv(file_name)

datasets = [
    "JapaneseVowels",
    "CatsDogs",
    "SpokenArabicDigits",
    "FSDD",
    "SPEECHCOMMANDS",
#    "MackeyGlass",
#    "Lorenz",
#    "Sunspot_daily",
]

data = data[data["dataset"].isin(datasets_keep)]
data["dataset"] = (data["dataset"]
                   .str.replace("SpokenArabicDigits", "Spoken\nArabic\nDigits")
                   .str.replace("SPEECHCOMMANDS",     "SPEECH\nCOMMANDS")
                   .str.replace("JapaneseVowels",     "Japanese\nVowels"))

data["Algorithm"] = data["function_name"].map(function_mapping)
datasets       = data["dataset"].unique()

# Apply the mapping to create a new column with descriptive labels
data['Algorithm'] = data['function_name'].map(function_mapping)

datasets = data['dataset'].unique()

# ------------------------------------------------------------------
# plotting parameters
# ------------------------------------------------------------------
metrics       = ["spectral_radius_mean", "pearson_mean", "CEV_mean", "dcor_mean"]
err_metrics   = ["spectral_radius_std",  "pearson_std", "CEV_std",  "dcor_std"]

fontsize = 20
width    = 0.10
x        = np.arange(len(datasets))

# ------------------------------------------------------------------
# create a figure will contains only the legend and  one bar-plot figure per metric (no legends inside)
# ------------------------------------------------------------------
legend_fig, legend_ax = plt.subplots(figsize=(14, 2))  # very flat
legend_ax.axis("off")                                  # no axes visible

for metric_index, metric in enumerate(metrics):
    fig, ax = plt.subplots(figsize=(14, 8))

    # -- bars -------------------------------------------------------
    for i, func in enumerate(functions_order):
        subset  = data[data["Algorithm"] == func]
        means   = subset[metric].values
        errors  = subset[err_metrics[metric_index]].values
        ax.bar(x + i*width, means, width,
               label=func,
               yerr=errors, capsize=5,
               color=function_colors[func],
               error_kw={"elinewidth": 2, "capthick": 2})

    # -- cosmetics --------------------------------------------------
    ax.set_ylabel(metric.replace("_", " ").title(), fontsize=fontsize)
    ax.tick_params(axis="y", labelsize=fontsize)
    ax.yaxis.set_major_locator(MaxNLocator(nbins=10))
    ax.set_xticks(x + width*(len(functions_order)-1)/2)
    ax.set_xticklabels(datasets, fontsize=fontsize)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    
    fig.tight_layout()          # no extra space needed for legend

    # after drawing the *first* metric, steal its handles for the legend
    if metric_index == 0:
        handles, labels = ax.get_legend_handles_labels()

# --- legend -------------------------------------------------
legend_fig.legend(handles, labels,
                  title="Algorithm",
                  ncol=len(functions_order),
                  loc="center",            # center of this tiny figure
                  fontsize=14, title_fontsize=14, frameon=False)

legend_fig.tight_layout()
plt.show()          # shows legend first, then the bar-plot figures

# Separability

In [ ]:
from scipy import stats
from numpy import random

# Evaluating
from performances.esn_model_evaluation import train_model_for_classification, predict_model_for_classification, compute_score
from performances.esn_model_evaluation import (train_model_for_prediction, init_reservoir, init_ip_reservoir, init_local_rule_reservoir, 
                                                init_ip_local_rule_reservoir, init_readout)
from analysis.richness import spectral_radius, pearson, squared_uncoupled_dynamics_alternative, distance_correlation


from importlib import reload
import analysis.separability
from analysis.separability import (
    inter_intra_class_distance,
    fisher_discriminant_ratio,
    silhouette,
    davies_bouldin,
    calinski_harabasz
)
reload(analysis.separability)

nb_jobs = 10
def evaluate_dataset_on_test_alternative(study, dataset_name, function_name, pretrain_data, train_data, test_data, Y_train, Y_test, is_instances_classification, nb_trials = 8, record_metrics=False):
    # Collect all hyperparameters in a dictionary
    hyperparams = {param_name: param_value for param_name, param_value in study.best_trial.params.items()}
    print(hyperparams)
    leaky_rate = 1
    input_connectivity = 1

    # score for prediction
    if dataset_name == "Sunspot":
        start_step = 30
        end_step = 500
    else:
        start_step = 500
        end_step = 1500
    SLICE_RANGE = slice(start_step, end_step)

    if 'variance_target' not in hyperparams and 'min_variance' in hyperparams:
        hyperparams['variance_target'] = hyperparams['min_variance']
    if not is_instances_classification:
        hyperparams['use_full_instance'] = False

    RIDGE_COEF = 10**hyperparams['ridge']
    
    if function_name in ["hadsp", "desp"]:
        max_partners = np.inf
    
    inter_dists = []
    intra_dists = []
    separability_ratios = []
    sil_scores = []
    dbi_scores = []
    ch_scores  = []
    fdr_scores = []

    for i in range(nb_trials):
        print("Trial", i + 1, "of", nb_trials)
        common_index = 1
        if is_instances_classification:
            common_size = pretrain_data[0].shape[common_index]
        else:
            common_size = pretrain_data.shape[common_index]

        # We want the size of the reservoir to be at least network_size
        K = math.ceil(hyperparams['network_size'] / common_size)
        n = common_size * K
        
        if function_name in ["diag_ee", "diag_ei"]:
            use_block = True
        else:
            use_block = False
            
        # UNSUPERVISED PRETRAINING 
        if function_name == "random_ee":
            Win, W, bias = init_matrices(n, input_connectivity, hyperparams['connectivity'],  K, w_distribution=stats.uniform(loc=0, scale=1), use_block=use_block, seed=random.randint(0, 1000))
        else:
            Win, W, bias = init_matrices(n, input_connectivity, hyperparams['connectivity'],  K, w_distribution=stats.uniform(loc=-1, scale=2), use_block=use_block, seed=random.randint(0, 1000))
        bias *= hyperparams['bias_scaling']
        Win *= hyperparams['input_scaling']

        if function_name == "hadsp":
            W, (_, _, _) = run_algorithm(W, Win, bias, hyperparams['leaky_rate'], activation_function, pretrain_data, 
                                     hyperparams['weight_increment'], hyperparams['target_rate'], hyperparams['rate_spread'], function_name, 
                                     multiple_instances=is_instances_classification, 
                                     min_increment = hyperparams['min_increment'], max_increment=hyperparams['max_increment'], use_full_instance=hyperparams['use_full_instance'],
                                     max_partners=max_partners, method="pearson", n_jobs=nb_jobs)
        elif function_name == "desp":
            print("DESP")
            W, (_, _, _) = run_algorithm(W, Win, bias, hyperparams['leaky_rate'], activation_function, pretrain_data, 
                                         hyperparams['weight_increment'], hyperparams['variance_target'], hyperparams['variance_spread'], function_name, 
                                         multiple_instances=is_instances_classification, 
                                         min_increment = hyperparams['min_increment'], max_increment=hyperparams['max_increment'], use_full_instance = hyperparams['use_full_instance'], 
                                         max_partners=max_partners, method = "pearson", 
                                         intrinsic_saturation=hyperparams['intrinsic_saturation'], intrinsic_coef=hyperparams['intrinsic_coef'], 
                                         n_jobs = nb_jobs)
        elif function_name in ["random_ee", "random_ei", "diag_ee", "diag_ei", "ip_correct", "anti-oja_fast", "ip-anti-oja_fast"]:
            eigen = sparse.linalg.eigs(W, k=1, which="LM", maxiter=W.shape[0] * 20, tol=0.1, return_eigenvectors=False)
            W *= hyperparams['spectral_radius'] / max(abs(eigen))
        else:
            raise ValueError(f"Invalid function: {function_name}")
        
        # unsupervised local rules
        if is_instances_classification:
            unsupervised_pretrain = np.concatenate(pretrain_data).astype(float)
        else:
            unsupervised_pretrain = pretrain_data.astype(float)
        if function_name == "ip_correct":
            reservoir = init_ip_reservoir(W, Win, bias, mu=hyperparams['mu'], sigma=hyperparams['sigma'], learning_rate=hyperparams['learning_rate'],
                                          leaking_rate=hyperparams['leaky_rate'], activation_function=activation_function
                                          )
            _ = reservoir.fit(unsupervised_pretrain, warmup=100)
        elif function_name == "anti-oja_fast":
            reservoir = init_local_rule_reservoir(W, Win, bias, local_rule="anti-oja", eta=hyperparams['oja_eta'],
                                                  synapse_normalization=False, bcm_theta=None,
                                                  leaking_rate=hyperparams['leaky_rate'], activation_function=activation_function,
                                                  )
            _ = reservoir.fit(unsupervised_pretrain, warmup=100)        
        elif function_name == "ip-anti-oja_fast":
            reservoir = init_ip_local_rule_reservoir(W, Win, bias, local_rule="anti-oja", eta=hyperparams['oja_eta'],
                                                      synapse_normalization=False, bcm_theta=None,
                                                      mu=hyperparams['mu'], sigma=hyperparams['sigma'], learning_rate=hyperparams['learning_rate'],
                                                      leaking_rate=hyperparams['leaky_rate'], activation_function=activation_function,
                                                      )
            _ = reservoir.fit(unsupervised_pretrain, warmup=100)
        else:
            reservoir = init_reservoir(W, Win, bias, leaky_rate, activation_function)
        readout = init_readout(ridge_coef=RIDGE_COEF)


        # TRAINING and EVALUATION
        # Step 1: Collect final hidden states
        final_states = []
        for seq in test_data:
            states = reservoir.run(seq)
            final_states.append(states[-1])

        final_states = np.array(final_states)  # (n_test, reservoir_dim)

        # Convert one-hot labels to flat labels
        if Y_test.ndim == 2:
            y_test = np.argmax(Y_test, axis=1)
        else:
            y_test = np.array(Y_test)

        # Step 2: Compute inter/intra class distances
        if is_instances_classification:
            inter_dist, intra_dist, sep_ratio = inter_intra_class_distance(final_states, y_test)
            sil = silhouette(final_states, y_test)
            dbi = davies_bouldin(final_states, y_test)
            ch  = calinski_harabasz(final_states, y_test)
            fdr = fisher_discriminant_ratio(final_states, y_test)

            inter_dists.append(inter_dist)
            intra_dists.append(intra_dist)
            separability_ratios.append(sep_ratio)

            sil_scores.append(sil)
            dbi_scores.append(dbi)
            ch_scores.append(ch)
            fdr_scores.append(fdr)

    return {
        'inter': inter_dists,
        'intra': intra_dists,
        'ratio': separability_ratios,
        'silhouette': sil_scores,
        'davies_bouldin': dbi_scores,
        'calinski_harabasz': ch_scores,
        'fisher_ratio': fdr_scores
    }


In [ ]:
from performances.utility import retrieve_best_model

datasets = [
    "JapaneseVowels",
    "CatsDogs",
    "FSDD",
    "SpokenArabicDigits",
    # "SPEECHCOMMANDS",
]


corr_columns = []
for dataset in datasets:
    print(dataset)
    pretrain_data, train_data, test_data, Y_train, Y_test, is_multivariate, \
        is_instances_classification = load_data(dataset, data_type, noise_std, visualize=False)

    for function_name in [
        "random_ee", "random_ei", "ip_correct",
        "anti-oja_fast", "ip-anti-oja_fast", "hadsp", "desp",
    ]:
        print(function_name)

        study = retrieve_best_model(
            function_name, dataset, is_multivariate,
            variate_type="multi", data_type="normal"
        )

        results = evaluate_dataset_on_test_alternative(
            study, dataset, function_name,
            pretrain_data, train_data, test_data,
            Y_train, Y_test,
            is_instances_classification,
            nb_trials=4,
            record_metrics=True
        )

        # add *one* dict to corr_rows
        corr_columns.append({
            "dataset": dataset,
            "function_name": function_name,
            "inter_dists_mean": np.mean(results['inter']),
            "inter_dists_std":  np.std(results['inter']),
            "intra_dists_mean": np.mean(results['intra']),
            "intra_dists_std":  np.std(results['intra']),
            "separability_ratios_mean": np.mean(results['ratio']),
            "separability_ratios_std":  np.std(results['ratio']),
            "silhouette_mean": np.mean(results['silhouette']),
            "silhouette_std":  np.std(results['silhouette']),
            "davies_bouldin_mean": np.mean(results['davies_bouldin']),
            "davies_bouldin_std":  np.std(results['davies_bouldin']),
            "calinski_harabasz_mean": np.mean(results['calinski_harabasz']),
            "calinski_harabasz_std":  np.std(results['calinski_harabasz']),
            "fisher_ratio_mean": np.mean(results['fisher_ratio']),
            "fisher_ratio_std":  np.std(results['fisher_ratio']),
        })


In [ ]:
file_name = "outputs/metrics.csv"

orig    = pd.read_csv(file_name).set_index(["dataset", "function_name"])
corr_df = pd.DataFrame(corr_columns).set_index(["dataset", "function_name"])

# 1) make sure orig has all the columns corr_df has (fills with NaN where missing)
for col in corr_df.columns:
    if col not in orig.columns:
        orig[col] = np.nan

# 2) now update in-place (only non-null values from corr_df overwrite orig)
orig.update(corr_df)

# 3) reset index & write out
augmented = orig.reset_index()
augmented.to_csv(file_name, index=False)
print(f"✔ Added / updated columns in {file_name}")

## Visualize

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# ------------------------------------------------------------------
# read & prepare the data  
# ------------------------------------------------------------------
data = pd.read_csv("outputs/metrics.csv")

datasets = [
    "JapaneseVowels", "CatsDogs", "SpokenArabicDigits",
    "FSDD", "SPEECHCOMMANDS"
]

data = data[data["dataset"].isin(datasets)]
data["dataset"] = (data["dataset"]
                   .str.replace("SpokenArabicDigits", "Spoken\nArabic\nDigits")
                   .str.replace("SPEECHCOMMANDS",     "SPEECH\nCOMMANDS")
                   .str.replace("JapaneseVowels",     "Japanese\nVowels"))

# … your mapping / palette …
data["Algorithm"] = data["function_name"].map(function_mapping)
datasets = data["dataset"].unique()

# ------------------------------------------------------------------
# parameters
# ------------------------------------------------------------------
metrics       = ["inter_dists_mean",  "intra_dists_mean",
                 "separability_ratios_mean", "silhouette_mean",
                 "davies_bouldin_mean", "calinski_harabasz_mean",
                 "fisher_ratio_mean"]

error_metrics = ["inter_dists_std",  "intra_dists_std",
                 "separability_ratios_std", "silhouette_std",
                 "davies_bouldin_std", "calinski_harabasz_std",
                 "fisher_ratio_std"]

fontsize = 20
width    = 0.10
x        = np.arange(len(datasets))

# ------------------------------------------------------------------
# create a legend figure and  bar-plot figures
# ------------------------------------------------------------------
legend_fig, legend_ax = plt.subplots(figsize=(14, 2))   # flat banner
legend_ax.axis("off")                                   # hide axes
handles = labels = None   # will be filled when we plot the first metric


for m, metric in enumerate(metrics):
    fig, ax = plt.subplots(figsize=(14, 8))

    # --- bars ------------------------------------------------------
    for i, func in enumerate(functions_order):
        subset  = data[data["Algorithm"] == func]
        means   = subset[metric].values
        errors  = subset[error_metrics[m]].values

        ax.bar(x + i*width, means, width,
               label=func,
               yerr=errors, capsize=5,
               color=function_colors[func],
               error_kw={"elinewidth": 2, "capthick": 2})

    # --- cosmetics -------------------------------------------------
    ax.set_ylabel(metric.replace("_", " ").title(), fontsize=fontsize)
    ax.tick_params(axis="y", labelsize=fontsize)
    ax.yaxis.set_major_locator(MaxNLocator(nbins=10))
    ax.set_xticks(x + width*(len(functions_order)-1)/2)
    ax.set_xticklabels(datasets, fontsize=fontsize)

    # remove top/right spines
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    # grab legend entries once (from the first metric figure)
    if m == 0:
        handles, labels = ax.get_legend_handles_labels()

# --- legend -------------------------------------------------
legend_fig.legend(handles, labels,
                  title="Algorithm",
                  ncol=len(functions_order),
                  loc="center",
                  fontsize=14, title_fontsize=14, frameon=False)

legend_fig.tight_layout()
plt.show()                      # shows the legend banner

# Final matrix

In [ ]:
from performances.utility import camel_to_snake, retrieve_best_model
from reservoir.reservoir import init_matrices
from connexion_generation.hag import run_algorithm
from scipy import sparse


# List of datasets
classification = [
    "JapaneseVowels",
    "CatsDogs",
    "FSDD",
    "SpokenArabicDigits",
    "SPEECHCOMMANDS",
]

prediction = [
    "MackeyGlass",
    "Lorenz",
    "Sunspot_daily",
]
datasets=prediction


# Initialize lists to store results and max values
Ws = []
titles = []
max_values = []

leaky_rate = 1
input_connectivity = 1

# Loop through datasets and function names to compute W matrices and find global vmax
for dataset in datasets:
    print(dataset)
    pretrain_data, train_data, test_data, Y_train, Y_test, is_multivariate, is_instances_classification = load_data(dataset, data_type, noise_std)

    for function_name in ["random_ee", "random_ei", "diag_ee", "diag_ei", "ip_correct", "anti-oja_fast", "ip-anti-oja_fast",  "hadsp", "desp"]:
        print(function_name)
        # Get the best trial from the study
        study = retrieve_best_model(function_name, dataset, is_multivariate, variate_type = "multi", data_type = "normal")
        hyperparams = {param_name: param_value for param_name, param_value in study.best_trial.params.items()}
        print(hyperparams)

        if 'variance_target' not in hyperparams and 'min_variance' in hyperparams:
            hyperparams['variance_target'] = hyperparams['min_variance']
        if not is_instances_classification:
            hyperparams['use_full_instance'] = False
    
        if function_name in ["hadsp", "desp"]:
            max_partners = np.inf
        
        common_index = 1
        if is_instances_classification:
            common_size = pretrain_data[0].shape[common_index]
        else:
            common_size = pretrain_data.shape[common_index]

        # We want the size of the reservoir to be at least network_size
        K = math.ceil(hyperparams["network_size"] / common_size)
        n = common_size * K
        
        if function_name in ["diag_ee", "diag_ei"]:
            use_block = True
        else:
            use_block = False
            
        # UNSUPERVISED PRETRAINING 
        if function_name == "random_ee":
            Win, W, bias = init_matrices(n, input_connectivity, hyperparams['connectivity'],  K, w_distribution=stats.uniform(loc=0, scale=1), use_block=use_block, seed=random.randint(0, 1000))
        else:
            Win, W, bias = init_matrices(n, input_connectivity, hyperparams['connectivity'],  K, w_distribution=stats.uniform(loc=-1, scale=2), use_block=use_block, seed=random.randint(0, 1000))
        bias *= hyperparams['bias_scaling']
        Win *= hyperparams['input_scaling']

        if function_name == "hadsp":
            W, (_, _, _) = run_algorithm(W, Win, bias, hyperparams['leaky_rate'], activation_function, pretrain_data, 
                                     hyperparams['weight_increment'], hyperparams['target_rate'], hyperparams['rate_spread'], function_name, 
                                     multiple_instances=is_instances_classification, 
                                     min_increment = hyperparams['min_increment'], max_increment=hyperparams['max_increment'], use_full_instance=hyperparams['use_full_instance'],
                                     max_partners=max_partners, method="pearson", n_jobs=nb_jobs)
        elif function_name == "desp":
            W, (_, _, _) = run_algorithm(W, Win, bias, hyperparams['leaky_rate'], activation_function, pretrain_data, 
                                         hyperparams['weight_increment'], hyperparams['variance_target'], hyperparams['variance_spread'], function_name, 
                                         multiple_instances=is_instances_classification, 
                                         min_increment = hyperparams['min_increment'], max_increment=hyperparams['max_increment'], use_full_instance = hyperparams['use_full_instance'], 
                                         max_partners=max_partners, method = "pearson", 
                                         intrinsic_saturation=hyperparams['intrinsic_saturation'], intrinsic_coef=hyperparams['intrinsic_coef'], 
                                         n_jobs = nb_jobs)
        elif function_name in ["random_ee", "random_ei"]:
            eigen = sparse.linalg.eigs(W, k=1, which="LM", maxiter=W.shape[0] * 20, tol=0.1, return_eigenvectors=False)
            W *= hyperparams['spectral_radius'] / max(abs(eigen))
            
        elif function_name in ["ip_correct", "anti-oja_fast", "ip-anti-oja_fast"]:
            eigen = sparse.linalg.eigs(W, k=1, which="LM", maxiter=W.shape[0] * 20, tol=0.1, return_eigenvectors=False)
            W *= hyperparams['spectral_radius'] / max(abs(eigen))

            # unsupervised local rules
            if is_instances_classification:
                unsupervised_pretrain = np.concatenate(pretrain_data).astype(float)
            else:
                unsupervised_pretrain = pretrain_data.astype(float)
            if function_name == "ip_correct":
                reservoir = init_ip_reservoir(W, Win, bias, mu=hyperparams['mu'], sigma=hyperparams['sigma'], learning_rate=hyperparams['learning_rate'],
                                              leaking_rate=hyperparams['leaky_rate'], activation_function=activation_function
                                              )
                _ = reservoir.fit(unsupervised_pretrain, warmup=100)
            elif function_name == "anti-oja_fast":
                reservoir = init_local_rule_reservoir(W, Win, bias, local_rule="anti-oja", eta=hyperparams['oja_eta'],
                                                       synapse_normalization=True, bcm_theta=None,
                                                       leaking_rate=hyperparams['leaky_rate'], activation_function=activation_function,
                                                       )
                _ = reservoir.fit(unsupervised_pretrain, warmup=100)
            elif function_name == "ip-anti-oja_fast":
                reservoir = init_ip_local_rule_reservoir(W, Win, bias, local_rule="anti-oja", eta=hyperparams['oja_eta'],
                                                          synapse_normalization=True, bcm_theta=None,
                                                          mu=hyperparams['mu'], sigma=hyperparams['sigma'], learning_rate=hyperparams['learning_rate'],
                                                          leaking_rate=hyperparams['leaky_rate'], activation_function=activation_function,
                                                          )
                _ = reservoir.fit(unsupervised_pretrain, warmup=100)
            else:
                reservoir = init_reservoir(W, Win, bias, leaky_rate, activation_function)

            W = reservoir.W
            

        # Store W matrix and corresponding title
        Ws.append(W)
        titles.append(f"{dataset} - {[function_name]}")
        max_values.append(np.max(W))


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

n_datasets = len(datasets)
n_functions = len(functions_order)

# 1) Determine global min/max to center color scale around 0
# Flatten all Ws into one array to find overall min & max
all_values = []
for W in Ws:
    if hasattr(W, "toarray"):
        W = W.toarray()
    all_values.append(W.ravel())
all_values = np.concatenate(all_values)

global_abs_max = np.max(np.abs(all_values))  # largest absolute value
g_vmin, g_vmax = -global_abs_max, global_abs_max

# 2) Create a figure with one row per dataset and one column per function
fig, axes = plt.subplots(
    n_datasets, 
    n_functions, 
    figsize=(n_functions * 3, n_datasets * 3),
    sharex=True, 
    sharey=True
)

# Ensure axes is 2D even if n_datasets=1 or n_functions=1
if n_datasets == 1:
    axes = np.array([axes])
if n_functions == 1:
    axes = np.array([axes]).T

idx = 0
im = None  # to store the last image for the colorbar
for i in range(n_datasets):
    for j in range(n_functions):
        W = Ws[idx]
        idx += 1
        
        # Convert to dense if sparse
        if hasattr(W, "toarray"):
            W = W.toarray()
        
        # 3) Plot base heatmap, using a diverging colormap like 'bwr'
        #    and a symmetric vmin/vmax around 0
        local_abs_max = np.max(np.abs(W))
        vmin, vmax = -local_abs_max, local_abs_max
        
        im = axes[i, j].imshow(
            W, 
            cmap='seismic', 
            interpolation='nearest', 
            vmin=vmin, 
            vmax=vmax
        )
        
        # 4) Overlay zeros in white
        zero_mask = (W == 0)
        axes[i, j].imshow(
            np.ma.masked_where(~zero_mask, W),
            cmap=mcolors.ListedColormap(['white']),
            interpolation='nearest'
        )
        
        # Remove per-subplot x/y labels
        axes[i, j].set_xlabel('')
        axes[i, j].set_ylabel('')
        
        # Row label: dataset name on the left edge
        if j == 0:
            axes[i, j].text(
                -0.3, 0.5, datasets[i],
                rotation=90,
                transform=axes[i, j].transAxes,
                ha='center',
                va='center',
                fontsize=12
            )
        
        # Column label: function name on top row
        if i == 0:
            axes[i, j].set_title(functions_order[j], fontsize=12)
            
        # 5) Add a colorbar for each heatmap
        #cbar = fig.colorbar(im, ax=axes[i, j], fraction=0.046, pad=0.04)
        #cbar.ax.tick_params(labelsize=10)
        
# 5) Add shared axis labels
fig.text(0.5, 0.04, 'Neurons', ha='center', va='center', fontsize=14)
fig.text(0.04, 0.5, 'Neurons', ha='center', va='center', rotation='vertical', fontsize=14)

# Tight layout
fig.tight_layout()

# Single colorbar on the right
cbar = fig.colorbar(im, ax=axes.ravel().tolist(), fraction=0.03, pad=0.04)
cbar.set_label('Value', fontsize=12)
plt.savefig(f'connectivity_matrices.png', dpi=300, bbox_inches='tight')
plt.show()